In [3]:
#load data
import pandas as pd
from scipy import sparse
from collections import defaultdict
from scipy import sparse
from sklearn import model_selection as cv
import numpy as np

header = ["reviewerId","productId","rating","time"]
df = pd.read_csv('ratings_Video_Games.csv', names=header)
#df = pd.read_csv('item_dedup.csv', names=header)
n_users = df.reviewerId.unique().shape[0]
n_items = df.productId.unique().shape[0]
print ('Number of users = ' + str(n_users) + ' | Number of items = ' + str(n_items))


train_data, test_data = cv.train_test_split(df, test_size=0)

# Create dict for product id 
#df.productId.unique()
productDict = defaultdict(dict)
userDict = defaultdict(dict)
count = 0


for id in train_data.reviewerId.unique():
    userDict[id] = count
    count += 1

count = 0
for id in train_data.productId.unique():
    productDict[id] = count
    count += 1
    
n_users = len(userDict)
n_items = len(productDict)

print(n_users, " ",  n_items,"\n")

mat = sparse.lil_matrix((n_users, n_items),dtype=np.float32)
        
print(productDict['B00004XOLO'])
print(userDict['A2EHIE51TERF2C'])


for line in train_data.itertuples():
    mat[userDict[line[1]],productDict[line[2]]] = line[3]

print(mat.getnnz)
print("done")

Number of users = 826767 | Number of items = 50210
826767   50210 

22941
307400
<bound method lil_matrix.getnnz of <826767x50210 sparse matrix of type '<class 'numpy.float32'>'
	with 1324753 stored elements in LInked List format>>
done


In [4]:
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.metrics.pairwise import cosine_similarity
import sklearn.preprocessing as pp

def cosine_similarities(mat):
    col_normed_mat = pp.normalize(mat, axis=0)
    print(col_normed_mat)
    return col_normed_mat.T * col_normed_mat

def user_cosine_similarities(mat):
    col_normed_mat = pp.normalize(mat, axis=0)
    #print(col_normed_mat)
    return col_normed_mat * col_normed_mat.T


#cosine_similarities(mat)
test = cosine_similarities(mat)
userMatrix = user_cosine_similarities(mat)
#user_similarity = pairwise_distances(mat, metric='cosine')
print("done")
print(test.getnnz)
print(userMatrix.getnnz)


#cx = sparse.coo_matrix(test)

#for i,j,v in zip(cx.row, cx.col, cx.data):
#        if v > 0.5:
#            print ("(%d, %d), %s" % (i,j,v))



  (0, 0)	0.046895154
  (259, 0)	0.009379031
  (870, 0)	0.046895154
  (986, 0)	0.046895154
  (1242, 0)	0.046895154
  (2196, 0)	0.037516125
  (2856, 0)	0.018758062
  (5246, 0)	0.046895154
  (6921, 0)	0.046895154
  (8465, 0)	0.046895154
  (9571, 0)	0.046895154
  (9725, 0)	0.046895154
  (10358, 0)	0.046895154
  (10421, 0)	0.018758062
  (10640, 0)	0.046895154
  (10745, 0)	0.046895154
  (10778, 0)	0.046895154
  (11012, 0)	0.046895154
  (11065, 0)	0.046895154
  (11282, 0)	0.046895154
  (11674, 0)	0.046895154
  (12391, 0)	0.046895154
  (12431, 0)	0.037516125
  (13569, 0)	0.037516125
  (14141, 0)	0.046895154
  :	:
  (242035, 50185)	1.0
  (382083, 50186)	1.0
  (635026, 50187)	1.0
  (825271, 50188)	1.0
  (825320, 50189)	1.0
  (20394, 50190)	1.0
  (1831, 50191)	1.0
  (825516, 50192)	1.0
  (433224, 50193)	1.0
  (825611, 50194)	1.0
  (214330, 50195)	1.0
  (825776, 50196)	1.0
  (150950, 50197)	1.0
  (825981, 50198)	1.0
  (826017, 50199)	1.0
  (35099, 50200)	1.0
  (9070, 50201)	1.0
  (826224, 50202)	1

In [5]:
# https://stackoverflow.com/questions/31790819/scipy-sparse-csr-matrix-how-to-get-top-ten-values-and-indices
#print(test[50208,40340])

row = test.getrow(6565).toarray()[0].ravel()
top_ten_indicies = row.argsort()[-6:]
top_ten_values = row[row.argsort()[-6:]]

print(top_ten_indicies)

print(top_ten_values)

#for x in top_ten_indicies:
#    print(list(productDict.keys())[list(productDict.values()).index(x)])
    


#for x in range(0,n_items):
#    print(x)
#    row = test.getrow(x).toarray()[0].ravel()
#    top_ten_values = (row[row.argsort()[-10:]])

print("done")



#array = test.getrow(i)
#print(array)


[44048 38045 48908 49963 40200  6565]
[0.11458731 0.11458731 0.11458731 0.11458731 0.11458731 1.0000007 ]
done


In [6]:
def getTopFiveSimiliarItems(itemId):
    row = test.getrow(itemId).toarray()[0].ravel()
    top_five_indicies = row.argsort()[-6:]
    top_five_values = row[row.argsort()[-6:]]
    print(top_five_values)
    print(top_five_indicies)
    for x in top_five_indicies:
        #ignore same item
        if (x != itemId):
            print(list(productDict.keys())[list(productDict.values()).index(x)])
        
getTopFiveSimiliarItems(9331)
getTopFiveSimiliarItems(productDict["B000EBC0PI"])
    

[0.05973976 0.0713971  0.07638597 0.14079697 0.14079697 1.        ]
[23380 41893 25419 44857 36698  9331]
B004Q4CZY0
B000S8IGP0
B002JTMNSW
B003O68TBY
B003JH8LD4
[0.         0.03449096 0.04490133 0.06150693 0.30411437 1.        ]
[16730   226  8782 10200 32779 27603]
B0041D8N2E
B004FYEZMQ
B0047O2Q0M
B000K8ZTK8
B0007PIE9C


In [7]:
print(df.loc[df['reviewerId'] == "A2UDD2XDPD80BH"])
print("A")
line = df.loc[df['reviewerId'] == "A2UDD2XDPD80BH"]

for x in line.iterrows(): #finding all items rated by the user
    #print(x[1][1])
    print(productDict[x[1][1]])
    getTopFiveSimiliarItems(productDict[x[1][1]])
    #print("======================")
    
#df.loc[df['reviewerId'] == "A10RFAF1CGFKHZ"]
print("AA")
print(list(productDict.keys())[list(productDict.values()).index(1076)], "\nB") 
print(list(productDict.keys())[list(productDict.values()).index(23340)], "\nC") 
print(list(userDict.keys())[list(userDict.values()).index(76565)], "\n") 

            reviewerId   productId  rating        time
124881  A2UDD2XDPD80BH  B000066TS5     1.0  1160956800
198242  A2UDD2XDPD80BH  B0001I1LD6     5.0  1145664000
204696  A2UDD2XDPD80BH  B00020LZAW     5.0  1142553600
228327  A2UDD2XDPD80BH  B00067AVT2     5.0  1142208000
237315  A2UDD2XDPD80BH  B0007PLKZM     5.0  1160092800
258699  A2UDD2XDPD80BH  B0009WPZM2     5.0  1159142400
270903  A2UDD2XDPD80BH  B000B43OXU     5.0  1144540800
284015  A2UDD2XDPD80BH  B000BI3QK2     5.0  1161648000
285552  A2UDD2XDPD80BH  B000BLM5PG     5.0  1156809600
289093  A2UDD2XDPD80BH  B000C427NM     5.0  1155427200
289669  A2UDD2XDPD80BH  B000CBVC0Y     5.0  1170979200
304679  A2UDD2XDPD80BH  B000FBMITA     5.0  1149206400
319037  A2UDD2XDPD80BH  B000FRS9II     5.0  1167436800
A
1054
[0.0552584  0.05794913 0.06040754 0.06185607 0.14857282 1.0000049 ]
[ 279 7000  525 6908 4845 1054]
B00008URUA
B001G3B0HC
B00005TNI6
B00068O278
B0009A4EVM
31702
[0.08253928 0.0927837  0.09755657 0.10920884 0.18762553 1.    

In [8]:
def countItemRatings(itemID): #counts the number of 1, 2, 3, 4, and 5 star ratings and returns an array of those totals (for a single ID)
    ratings = [0, 0, 0, 0, 0] #1,2,3,4,5 (so that if you want the number of n star ratings, you look at index n-1)
    reviewList = df.loc[df['productId'] == itemID]
    print( reviewList, "\n_______\n")
    for item in reviewList.iterrows():
        itemRating = item[1][2]
        if( itemRating == 5):
            ratings[4]+=1
        elif( itemRating == 4):
            ratings[3]+=1
        elif( itemRating == 3):
            ratings[2]+=1
        elif( itemRating == 2):
            ratings[1]+=1
        elif( itemRating == 1):
            ratings[0]+=1
    print( ratings)
    #print(list(productDict.keys())[list(productDict.values()).index(itemId)])
    

In [9]:
countItemRatings("B000066TS5")

            reviewerId   productId  rating        time
124691  A133TLIM50S73O  B000066TS5     5.0  1405468800
124692  A3AAT9AJ6ERNNA  B000066TS5     5.0  1405123200
124693  A25FGD52IVQ5DO  B000066TS5     5.0  1405296000
124694  A32VZMBDEG9HXY  B000066TS5     4.0  1055376000
124695  A1QQETPCQVDBFR  B000066TS5     5.0  1088726400
124696   AW7ROQ25QKACW  B000066TS5     4.0  1085270400
124697  A3LW6FZ12WUX2A  B000066TS5     1.0  1038873600
124698   AIDBAXSZ5ZMEH  B000066TS5     5.0  1124236800
124699   AFOVYN9DVKDQU  B000066TS5     5.0  1341360000
124700   A5QKU60PFZWMV  B000066TS5     5.0  1389052800
124701  A1BD8I14BUFVJ8  B000066TS5     4.0  1218067200
124702  A219WFI4NFP6D8  B000066TS5     2.0  1380672000
124703  A3VQUCRSSVQJYV  B000066TS5     5.0  1215820800
124704  A29UWODNPLIACN  B000066TS5     5.0  1046044800
124705  A35EIPKVVOI9TL  B000066TS5     5.0  1042588800
124706    A5BVWSS43142  B000066TS5     5.0  1049500800
124707   AY9V1PY5VSY5U  B000066TS5     5.0  1038528000
124708  A2